In [1]:
import os
%pwd

'/Users/vision8/Desktop/ML/Chicken Disease Classification/Chicken_disease_classification/research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'/Users/vision8/Desktop/ML/Chicken Disease Classification/Chicken_disease_classification'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [5]:
%pip install --upgrade ensure

from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories



Note: you may need to restart the kernel to use updated packages.


In [6]:
from cnnClassifier.constants import *

CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")

class ConfigurationManager :
    def __init__(
        self,
        config_file_path: CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH):
    
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

        

In [7]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size



In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
                )
            logger.info(f"Downloaded {filename} to {self.config.local_data_file} \n {headers}")
        else:
            logger.info(f"{self.config.local_data_file} already exists, size: {get_size(self.config.local_data_file)}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)
        


In [10]:
from pathlib import Path
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH

# Convert string paths to pathlib.Path objects
config_file_path = Path(CONFIG_FILE_PATH)
params_file_path = Path(PARAMS_FILE_PATH)

try:
    config = ConfigurationManager(config_file_path=config_file_path, params_file_path=params_file_path)
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2024-04-13 16:03:36,718: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-04-13 16:03:36,721: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-13 16:03:36,722: INFO: common: created directory at: artifacts]
[2024-04-13 16:03:36,723: INFO: common: created directory at: artifacts/data_ingestion]
[2024-04-13 16:03:39,613: INFO: 2443765520: Downloaded artifacts/data_ingestion/data.zip to artifacts/data_ingestion/data.zip 
 Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: E8A2:1F4FF0:33BA8B3:366DBCF:661A90B4
Accept-Ranges: bytes
Date: Sat, 13 Apr 2024 14:03:37 GMT
Via: 1.1 varnish
X-Served-By: cache-fr